# High/Low Breakout Trading Strategy with VectorBT

This notebook implements a trading strategy based on price breakouts:
- **Buy Signal**: Price exceeds the highest high of the last 5 minutes
- **Sell Signal**: Price falls below the lowest low of the last 5 minutes  
- **Position Management**: Close all positions at the end of each trading day

We'll use vectorbt for efficient backtesting and analysis.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import vectorbt as vbt
import matplotlib.pyplot as plt
from datetime import datetime, time
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully!")
print(f"VectorBT version: {vbt.__version__}")

Libraries imported successfully!
VectorBT version: 0.28.0


## Data Preparation

First, let's create sample data based on the format provided in the requirements.

In [19]:
# Create sample data based on the provided format
sample_data = """
Date,Minute,Open,Low,High,Close,Volume
2025-01-02,390,589.3900,588.4600,589.4500,589.4500,1128698.00
2025-01-02,391,589.4500,589.0400,589.6000,589.5700,151034.00
2025-01-02,392,589.4900,589.0319,589.6500,589.0900,96284.00
2025-01-02,393,589.0900,587.9800,589.1800,588.0100,98628.00
2025-01-02,394,587.9900,587.8100,588.5200,587.8200,98190.00
2025-01-02,395,587.8400,586.9100,587.9100,586.9100,127238.00
2025-01-02,396,586.9100,586.9100,587.5500,587.0500,163237.00
2025-01-02,397,587.0300,586.7100,587.3400,587.2210,185067.00
2025-01-02,398,587.2399,586.8900,587.5000,586.9300,146477.00
2025-01-02,399,586.9300,586.2300,587.0000,586.2500,135366.00
"""

# If you have actual data file, replace this with:
df = pd.read_csv('./../spy_minute.csv')

# from io import StringIO
# df = pd.read_csv(StringIO(sample_data.strip()))

print("Sample data loaded:")
print(df.head())
print(f"\nData shape: {df.shape}")

Sample data loaded:
         Date   Minuite    Open       Low    High   Close     Volume
0  2025-01-02       390  589.39  588.4600  589.45  589.45  1128698.0
1  2025-01-02       391  589.45  589.0400  589.60  589.57   151034.0
2  2025-01-02       392  589.49  589.0319  589.65  589.09    96284.0
3  2025-01-02       393  589.09  587.9800  589.18  588.01    98628.0
4  2025-01-02       394  587.99  587.8100  588.52  587.82    98190.0

Data shape: (52626, 7)


In [5]:
# For demonstration, let's generate more realistic historical data
# This simulates minute-by-minute data over multiple days

np.random.seed(42)  # For reproducible results

# Generate dates for multiple trading days (assuming 390 minutes per trading day)
start_date = pd.Timestamp('2025-01-02')
end_date = pd.Timestamp('2025-01-10')
trading_days = pd.bdate_range(start_date, end_date)

# Minutes from market open (assuming 9:30 AM start, minute 390 = 9:30 AM)
minutes_per_day = range(390, 780)  # 390 minutes = 6.5 hours trading day

# Generate synthetic OHLC data
data_list = []
base_price = 589.0
current_price = base_price

for date in trading_days:
    # Reset price with some overnight gap
    overnight_gap = np.random.normal(0, 2)  # Small overnight gap
    current_price += overnight_gap
    
    for minute in minutes_per_day:
        # Generate minute-level price movement
        price_change = np.random.normal(0, 0.5)  # Small random walk
        
        open_price = current_price
        close_price = current_price + price_change
        
        # Generate high and low based on volatility
        volatility = abs(np.random.normal(0, 0.3))
        high_price = max(open_price, close_price) + volatility
        low_price = min(open_price, close_price) - volatility
        
        # Generate volume
        volume = np.random.lognormal(11, 0.5)  # Log-normal distribution for volume
        
        data_list.append({
            'Date': date.strftime('%Y-%m-%d'),
            'Minute': minute,
            'Open': round(open_price, 4),
            'Low': round(low_price, 4),
            'High': round(high_price, 4),
            'Close': round(close_price, 4),
            'Volume': round(volume, 2)
        })
        
        current_price = close_price

# Create DataFrame
df = pd.DataFrame(data_list)

print(f"Generated synthetic data with {len(df)} rows")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print("\nFirst few rows:")
print(df.head(10))

Generated synthetic data with 2730 rows
Date range: 2025-01-02 to 2025-01-10

First few rows:
         Date  Minute      Open       Low      High     Close     Volume
0  2025-01-02     390  589.9934  589.7300  590.1877  589.9243  128221.55
1  2025-01-02     391  589.9243  589.7370  589.9945  589.8072  131874.55
2  2025-01-02     392  589.8072  589.6664  590.3318  590.1909   78533.46
3  2025-01-02     393  590.1909  589.8195  590.3307  589.9592   67574.17
4  2025-01-02     394  589.9592  588.4851  590.4767  589.0026   45200.17
5  2025-01-02     395  589.0026  588.4019  589.0969  588.4962   38024.58
6  2025-01-02     396  588.4962  587.3503  588.9359  587.7900   53482.61
7  2025-01-02     397  587.7900  587.3626  588.2512  587.8238   45606.64
8  2025-01-02     398  587.8238  587.4785  588.2245  587.8792   72247.21
9  2025-01-02     399  587.8792  587.4914  587.9668  587.5789   44318.02


In [6]:
# Create proper datetime index
df['DateTime'] = pd.to_datetime(df['Date']) + pd.to_timedelta((df['Minute'] - 390), unit='m')
df.set_index('DateTime', inplace=True)

# Sort by datetime to ensure proper order
df.sort_index(inplace=True)

print("Data with DateTime index:")
print(df.head())
print(f"\nIndex info: {df.index.min()} to {df.index.max()}")

Data with DateTime index:
                           Date  Minute      Open       Low      High  \
DateTime                                                                
2025-01-02 00:00:00  2025-01-02     390  589.9934  589.7300  590.1877   
2025-01-02 00:01:00  2025-01-02     391  589.9243  589.7370  589.9945   
2025-01-02 00:02:00  2025-01-02     392  589.8072  589.6664  590.3318   
2025-01-02 00:03:00  2025-01-02     393  590.1909  589.8195  590.3307   
2025-01-02 00:04:00  2025-01-02     394  589.9592  588.4851  590.4767   

                        Close     Volume  
DateTime                                  
2025-01-02 00:00:00  589.9243  128221.55  
2025-01-02 00:01:00  589.8072  131874.55  
2025-01-02 00:02:00  590.1909   78533.46  
2025-01-02 00:03:00  589.9592   67574.17  
2025-01-02 00:04:00  589.0026   45200.17  

Index info: 2025-01-02 00:00:00 to 2025-01-10 06:29:00


## Strategy Implementation

Now let's implement the high/low breakout strategy:
1. Calculate rolling 5-minute high and low
2. Generate buy signals when price breaks above 5-minute high
3. Generate sell signals when price breaks below 5-minute low
4. Close positions at end of day

In [7]:
# Calculate rolling 5-minute high and low
lookback_periods = 5

# Rolling maximum of High prices over last 5 periods
df['Rolling_High_5'] = df['High'].rolling(window=lookback_periods, min_periods=lookback_periods).max()

# Rolling minimum of Low prices over last 5 periods  
df['Rolling_Low_5'] = df['Low'].rolling(window=lookback_periods, min_periods=lookback_periods).min()

# Shift by 1 to avoid look-ahead bias (use previous 5-minute high/low)
df['Prev_Rolling_High_5'] = df['Rolling_High_5'].shift(1)
df['Prev_Rolling_Low_5'] = df['Rolling_Low_5'].shift(1)

print("Rolling high/low calculations completed:")
print(df[['High', 'Low', 'Close', 'Rolling_High_5', 'Rolling_Low_5', 'Prev_Rolling_High_5', 'Prev_Rolling_Low_5']].head(10))

Rolling high/low calculations completed:
                         High       Low     Close  Rolling_High_5  \
DateTime                                                            
2025-01-02 00:00:00  590.1877  589.7300  589.9243             NaN   
2025-01-02 00:01:00  589.9945  589.7370  589.8072             NaN   
2025-01-02 00:02:00  590.3318  589.6664  590.1909             NaN   
2025-01-02 00:03:00  590.3307  589.8195  589.9592             NaN   
2025-01-02 00:04:00  590.4767  588.4851  589.0026        590.4767   
2025-01-02 00:05:00  589.0969  588.4019  588.4962        590.4767   
2025-01-02 00:06:00  588.9359  587.3503  587.7900        590.4767   
2025-01-02 00:07:00  588.2512  587.3626  587.8238        590.4767   
2025-01-02 00:08:00  588.2245  587.4785  587.8792        590.4767   
2025-01-02 00:09:00  587.9668  587.4914  587.5789        589.0969   

                     Rolling_Low_5  Prev_Rolling_High_5  Prev_Rolling_Low_5  
DateTime                                            

In [8]:
# Generate trading signals

# Buy signal: Current high > Previous 5-minute rolling high
df['Buy_Signal'] = df['High'] > df['Prev_Rolling_High_5']

# Sell signal: Current low < Previous 5-minute rolling low
df['Sell_Signal'] = df['Low'] < df['Prev_Rolling_Low_5']

# Create end-of-day marker (assuming market closes at minute 779)
df['End_Of_Day'] = df['Minute'] == 779

print("Trading signals generated:")
print(f"Total buy signals: {df['Buy_Signal'].sum()}")
print(f"Total sell signals: {df['Sell_Signal'].sum()}")
print(f"End of day markers: {df['End_Of_Day'].sum()}")

# Show some examples of signals
signal_examples = df[df['Buy_Signal'] | df['Sell_Signal']].head(10)
print("\nFirst 10 trading signals:")
print(signal_examples[['Date', 'Minute', 'High', 'Low', 'Close', 'Prev_Rolling_High_5', 'Prev_Rolling_Low_5', 'Buy_Signal', 'Sell_Signal']])

Trading signals generated:
Total buy signals: 655
Total sell signals: 727
End of day markers: 7

First 10 trading signals:
                           Date  Minute      High       Low     Close  \
DateTime                                                                
2025-01-02 00:05:00  2025-01-02     395  589.0969  588.4019  588.4962   
2025-01-02 00:06:00  2025-01-02     396  588.9359  587.3503  587.7900   
2025-01-02 00:11:00  2025-01-02     401  589.2826  588.1388  588.9163   
2025-01-02 00:12:00  2025-01-02     402  589.3148  587.5380  587.9365   
2025-01-02 00:16:00  2025-01-02     406  588.0221  586.9461  587.0433   
2025-01-02 00:17:00  2025-01-02     407  587.2268  586.5214  586.7049   
2025-01-02 00:18:00  2025-01-02     408  587.4223  586.4531  587.1705   
2025-01-02 00:21:00  2025-01-02     411  588.0565  586.8365  587.6496   
2025-01-02 00:22:00  2025-01-02     412  588.2598  587.5411  588.1514   
2025-01-02 00:23:00  2025-01-02     413  588.7935  587.6899  588.3321   



## VectorBT Portfolio Simulation

Now let's use VectorBT to simulate the trading strategy and analyze performance.

In [9]:
# Prepare data for VectorBT
close_prices = df['Close']
buy_signals = df['Buy_Signal']
sell_signals = df['Sell_Signal']
end_of_day_signals = df['End_Of_Day']

# Create entries and exits for VectorBT
# Entry: Buy signal
entries = buy_signals

# Exit: Sell signal OR end of day
exits = sell_signals | end_of_day_signals

print(f"Entries (buy signals): {entries.sum()}")
print(f"Exits (sell signals + EOD): {exits.sum()}")

# Ensure we have some signals to work with
if entries.sum() == 0:
    print("Warning: No buy signals generated. Adjusting strategy parameters...")
    # Create more aggressive signals for demonstration
    df['Buy_Signal_Adjusted'] = df['Close'] > df['Close'].rolling(3).mean()
    df['Sell_Signal_Adjusted'] = df['Close'] < df['Close'].rolling(3).mean()
    entries = df['Buy_Signal_Adjusted']
    exits = df['Sell_Signal_Adjusted'] | end_of_day_signals
    print(f"Adjusted entries: {entries.sum()}")
    print(f"Adjusted exits: {exits.sum()}")

Entries (buy signals): 655
Exits (sell signals + EOD): 732


In [10]:
# Create VectorBT portfolio
portfolio = vbt.Portfolio.from_signals(
    close=close_prices,
    entries=entries,
    exits=exits,
    init_cash=100000,  # Starting with $100,000
    fees=0.001,        # 0.1% transaction fees
    freq='1min'        # Minute-level data
)

print("Portfolio created successfully!")
print(f"Total trades: {portfolio.trades.count()}")

Portfolio created successfully!
Total trades: 150


## Performance Analysis

In [11]:
# Portfolio statistics
stats = portfolio.stats()
print("=== Portfolio Performance Statistics ===")
print(stats)

=== Portfolio Performance Statistics ===
Start                               2025-01-02 00:00:00
End                                 2025-01-10 06:29:00
Period                                  1 days 21:30:00
Start Value                                    100000.0
End Value                                  69025.452354
Total Return [%]                             -30.974548
Benchmark Return [%]                         -11.133412
Max Gross Exposure [%]                            100.0
Total Fees Paid                            25168.044916
Max Drawdown [%]                              30.974548
Max Drawdown Duration                   1 days 21:19:00
Total Trades                                        150
Total Closed Trades                                 150
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  13.333333
Best Trade [%]                                 1.320673
Worst T

In [18]:
# Trade analysis
trades = portfolio.trades
print("=== Trade Analysis ===")
print(f"Total number of trades: {trades.count()}")
print(f"Win rate: {trades.win_rate * 100:.2f}%")

# Get trade returns safely
trade_returns = trades.returns
if len(trade_returns) > 0:
    print(f"Average trade return: {trade_returns.mean() * 100:.4f}%")
    
    # Filter winning and losing trades
    winning_trades = trade_returns[trade_returns > 0]
    losing_trades = trade_returns[trade_returns < 0]
    
    if len(winning_trades) > 0:
        print(f"Average winning trade: {winning_trades.mean() * 100:.4f}%")
    else:
        print("Average winning trade: No winning trades")
        
    if len(losing_trades) > 0:
        print(f"Average losing trade: {losing_trades.mean() * 100:.4f}%")
    else:
        print("Average losing trade: No losing trades")
        
    print(f"Best trade: {trade_returns.max() * 100:.4f}%")
    print(f"Worst trade: {trade_returns.min() * 100:.4f}%")
else:
    print("No trade returns available")

if trades.count() > 0:
    print("\n=== First 10 Trades ===")
    print(trades.records_readable.head(10))

=== Trade Analysis ===
Total number of trades: 150


TypeError: unsupported operand type(s) for *: 'method' and 'int'

In [ ]:
# Plotting results
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Plot 1: Price chart with signals
axes[0].plot(df.index, df['Close'], label='Close Price', linewidth=1, alpha=0.8)
axes[0].plot(df.index, df['Prev_Rolling_High_5'], label='5-Min Rolling High', linestyle='--', alpha=0.7)
axes[0].plot(df.index, df['Prev_Rolling_Low_5'], label='5-Min Rolling Low', linestyle='--', alpha=0.7)

# Mark buy and sell signals
buy_points = df[entries]
sell_points = df[exits]

if len(buy_points) > 0:
    axes[0].scatter(buy_points.index, buy_points['Close'], color='green', marker='^', s=50, label='Buy Signal', alpha=0.8)
if len(sell_points) > 0:
    axes[0].scatter(sell_points.index, sell_points['Close'], color='red', marker='v', s=50, label='Sell Signal', alpha=0.8)

axes[0].set_title('Price Chart with Trading Signals')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Portfolio value over time
portfolio_value = portfolio.value()
axes[1].plot(portfolio_value.index, portfolio_value.values, label='Portfolio Value', color='blue', linewidth=2)
axes[1].axhline(y=100000, color='gray', linestyle='--', alpha=0.7, label='Initial Capital')
axes[1].set_title('Portfolio Value Over Time')
axes[1].set_ylabel('Portfolio Value ($)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Cumulative returns
cumulative_returns = portfolio.total_return()
axes[2].plot(portfolio_value.index, (portfolio_value / 100000 - 1) * 100, label='Cumulative Return (%)', color='purple', linewidth=2)
axes[2].axhline(y=0, color='gray', linestyle='-', alpha=0.7)
axes[2].set_title('Cumulative Returns')
axes[2].set_ylabel('Return (%)')
axes[2].set_xlabel('Date')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Portfolio Value: ${portfolio.value().iloc[-1]:,.2f}")
print(f"Total Return: {portfolio.total_return() * 100:.2f}%")

## Risk Analysis

In [ ]:
# Calculate additional risk metrics
returns = portfolio.returns()
print("=== Risk Analysis ===")
print(f"Volatility (annualized): {returns.std() * np.sqrt(252 * 390) * 100:.2f}%")  # 252 trading days, 390 minutes per day
print(f"Maximum Drawdown: {portfolio.max_drawdown() * 100:.2f}%")
print(f"Sharpe Ratio: {portfolio.sharpe_ratio():.2f}")
print(f"Calmar Ratio: {portfolio.calmar_ratio():.2f}")

# Drawdown analysis
drawdowns = portfolio.drawdowns
if drawdowns.count() > 0:
    print(f"\nNumber of drawdown periods: {drawdowns.count()}")
    print(f"Average drawdown duration: {drawdowns.avg_duration}")
    print(f"Maximum drawdown duration: {drawdowns.max_duration}")

In [ ]:
# Plot drawdown chart
plt.figure(figsize=(15, 6))
drawdown_series = portfolio.drawdown()
plt.fill_between(drawdown_series.index, drawdown_series.values * 100, 0, alpha=0.3, color='red')
plt.plot(drawdown_series.index, drawdown_series.values * 100, color='red', linewidth=1)
plt.title('Portfolio Drawdowns')
plt.ylabel('Drawdown (%)')
plt.xlabel('Date')
plt.grid(True, alpha=0.3)
plt.show()

## Strategy Optimization and Variations

Let's test different lookback periods to see if we can improve performance.

In [ ]:
# Test different lookback periods
lookback_periods_to_test = [3, 5, 7, 10, 15]
results = []

for lookback in lookback_periods_to_test:
    # Calculate rolling high/low for this lookback period
    rolling_high = df['High'].rolling(window=lookback, min_periods=lookback).max().shift(1)
    rolling_low = df['Low'].rolling(window=lookback, min_periods=lookback).min().shift(1)
    
    # Generate signals
    buy_sig = df['High'] > rolling_high
    sell_sig = df['Low'] < rolling_low
    
    # Create portfolio
    try:
        pf = vbt.Portfolio.from_signals(
            close=close_prices,
            entries=buy_sig,
            exits=sell_sig | end_of_day_signals,
            init_cash=100000,
            fees=0.001,
            freq='1min'
        )
        
        results.append({
            'Lookback': lookback,
            'Total_Return': pf.total_return() * 100,
            'Sharpe_Ratio': pf.sharpe_ratio(),
            'Max_Drawdown': pf.max_drawdown() * 100,
            'Total_Trades': pf.trades.count(),
            'Win_Rate': pf.trades.win_rate * 100
        })
    except Exception as e:
        print(f"Error with lookback {lookback}: {e}")
        continue

# Create results DataFrame
optimization_results = pd.DataFrame(results)
print("=== Strategy Optimization Results ===")
print(optimization_results.round(2))

if len(optimization_results) > 0:
    best_lookback = optimization_results.loc[optimization_results['Sharpe_Ratio'].idxmax(), 'Lookback']
    print(f"\nBest performing lookback period: {best_lookback} minutes")

## Summary and Conclusions

In [ ]:
print("=== TRADING STRATEGY SUMMARY ===")
print("Strategy: High/Low Breakout")
print("- Buy when price breaks above 5-minute rolling high")
print("- Sell when price breaks below 5-minute rolling low")
print("- Close all positions at end of day")
print()
print("Key Performance Metrics:")
print(f"- Total Return: {portfolio.total_return() * 100:.2f}%")
print(f"- Sharpe Ratio: {portfolio.sharpe_ratio():.2f}")
print(f"- Maximum Drawdown: {portfolio.max_drawdown() * 100:.2f}%")
print(f"- Total Trades: {portfolio.trades.count()}")
print(f"- Win Rate: {portfolio.trades.win_rate * 100:.2f}%")
print(f"- Final Portfolio Value: ${portfolio.value().iloc[-1]:,.2f}")
print()
print("Next Steps:")
print("1. Load your actual historical data")
print("2. Adjust parameters based on optimization results")
print("3. Consider additional filters (volume, volatility, etc.)")
print("4. Implement position sizing rules")
print("5. Add stop-loss and take-profit levels")

## Data Loading Template

To use your actual data, replace the synthetic data generation with:

In [ ]:
# Template for loading your actual data
# Uncomment and modify this section when you have your data file

"""
# Load your actual data
df_actual = pd.read_csv('your_stock_data.csv')

# Convert to proper datetime format
df_actual['DateTime'] = pd.to_datetime(df_actual['Date']) + pd.to_timedelta((df_actual['Minute'] - 390), unit='m')
df_actual.set_index('DateTime', inplace=True)
df_actual.sort_index(inplace=True)

# Apply the same strategy logic
lookback_periods = 5
df_actual['Rolling_High_5'] = df_actual['High'].rolling(window=lookback_periods, min_periods=lookback_periods).max()
df_actual['Rolling_Low_5'] = df_actual['Low'].rolling(window=lookback_periods, min_periods=lookback_periods).min()
df_actual['Prev_Rolling_High_5'] = df_actual['Rolling_High_5'].shift(1)
df_actual['Prev_Rolling_Low_5'] = df_actual['Rolling_Low_5'].shift(1)

df_actual['Buy_Signal'] = df_actual['High'] > df_actual['Prev_Rolling_High_5']
df_actual['Sell_Signal'] = df_actual['Low'] < df_actual['Prev_Rolling_Low_5']
df_actual['End_Of_Day'] = df_actual['Minute'] == 779  # Adjust based on your data

# Run portfolio simulation
portfolio_actual = vbt.Portfolio.from_signals(
    close=df_actual['Close'],
    entries=df_actual['Buy_Signal'],
    exits=df_actual['Sell_Signal'] | df_actual['End_Of_Day'],
    init_cash=100000,
    fees=0.001,
    freq='1min'
)

print(portfolio_actual.stats())
"""

print("Data loading template provided above.")
print("Modify the file path and column names to match your actual data.")